### This notebook calculates two things:
1. Number of cars parked "at" a meter in a given time bucket
2. A pseudo-capacity for each meter

In [1]:
import pandas as pd
import math
import numpy as np
import datetime
import time
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import os
import shutil

In [2]:
#create dict of all meters mapped to their respective zones
meter_df = pd.read_csv("../data/meters.csv")
meter_dict = {}

for index, row in meter_df.iterrows():
    if row['status'] != 'Active':
        continue
        
    meter_dict[row['id']] = row['zone']

meter_dict

{'421009-CEDAR-0302': '421 - NorthSide',
 '409242-NDTHRG0106': '409 - Oakland 3',
 'PBP412': '412 - East Liberty',
 '426002-CENTRE1802': '426 - Hill District',
 '407118-MEYRAN0203': '407 - Oakland 1',
 '402403-BLVDAL0004': '402 - Downtown 2',
 '415018-ECARSN0701': '415 - SS & SSW',
 '421054-WESTRN0802': '421 - NorthSide',
 '415039-ECARSN2004': '415 - SS & SSW',
 '404470-PENAVE2001': '404 - Strip Disctrict',
 '401350-SMTHFD0601': '401 - Downtown 1',
 '411603-ELSWTH5805': '411 - Shadyside',
 '402505-ROSSST0101': '402 - Downtown 2',
 '404455-PENAVE1201': '404 - Strip Disctrict',
 'PBP415': '415 - SS & SSW',
 '421020-EOHIO-0502': '421 - NorthSide',
 '413028-MURRAY1903': '413 - Squirrel Hill',
 '415102-S27TH-0004': '415 - SS & SSW',
 '402407-BLVDAL0203': '402 - Downtown 2',
 '421069-CHLDWY0002': '421 - NorthSide',
 '407133-LTHROP0203': '407 - Oakland 1',
 '405009-BUTLER4302': '405 - Lawrenceville',
 '412593-TRADE-5901': '412 - East Liberty',
 '420013-GRANDV0202': '420 - Mt. Washington',
 '4

In [3]:
#create a base dataframe for a zone containing: start | day_of_week | date | month | time | is_holiday
start = '2018-01-01 00:00'
end = '2018-12-31 23:50'

#tz is VERY IMPORTANT or else daylight savings will ruin your day
dr = pd.date_range(start=start, end=end, freq='10min', tz='US/Eastern')
buckets = dr.to_frame(index=False, name='start')

buckets['start'] = pd.to_datetime(buckets['start'])
buckets['day_of_week'] = buckets['start'].dt.day_name()
buckets['date'] = buckets['start'].dt.strftime('%Y-%m-%d')
buckets['month'] = buckets['start'].dt.strftime('%m')
buckets['time'] = buckets['start'].dt.time

cal = calendar()
holidays = cal.holidays('2018-01-01', '2018-12-31')
buckets['is_holiday'] = pd.to_datetime(buckets['date']).dt.date.astype('datetime64').isin(holidays)

buckets

,start,day_of_week,date,month,time,is_holiday
0,2018-01-01 00:00:00-05:00,Monday,2018-01-01,01,00:00:00,True
1,2018-01-01 00:10:00-05:00,Monday,2018-01-01,01,00:10:00,True
2,2018-01-01 00:20:00-05:00,Monday,2018-01-01,01,00:20:00,True
3,2018-01-01 00:30:00-05:00,Monday,2018-01-01,01,00:30:00,True
4,2018-01-01 00:40:00-05:00,Monday,2018-01-01,01,00:40:00,True
...,...,...,...,...,...,...
52555,2018-12-31 23:10:00-05:00,Monday,2018-12-31,12,23:10:00,False
52556,2018-12-31 23:20:00-05:00,Monday,2018-12-31,12,23:20:00,False
52557,2018-12-31 23:30:00-05:00,Monday,2018-12-31,12,23:30:00,False
52558,2018-12-31 23:40:00-05:00,Monday,2018-12-31,12,23:40:00,False


In [4]:
#convert the starter dataframe into a unique dataframe for each meter
meter_buckets = {}
for meter in meter_dict:
        df = pd.DataFrame(buckets)
        df['used_spaces'] = 0
        df['meter'] = meter
        df['zone'] = meter_dict[meter]
        
        cols = df.columns.tolist()
        cols = cols[-2:] + cols[:-2]
        meter_buckets[meter] = df[cols] #I like having the zone on the left 
        
#printing an arbitrary zone just to see what we're expecting the dataframe to look like for each zone
meter_buckets['421009-CEDAR-0302']


,meter,zone,start,day_of_week,date,month,time,is_holiday,used_spaces
0,421009-CEDAR-0302,421 - NorthSide,2018-01-01 00:00:00-05:00,Monday,2018-01-01,01,00:00:00,True,0
1,421009-CEDAR-0302,421 - NorthSide,2018-01-01 00:10:00-05:00,Monday,2018-01-01,01,00:10:00,True,0
2,421009-CEDAR-0302,421 - NorthSide,2018-01-01 00:20:00-05:00,Monday,2018-01-01,01,00:20:00,True,0
3,421009-CEDAR-0302,421 - NorthSide,2018-01-01 00:30:00-05:00,Monday,2018-01-01,01,00:30:00,True,0
4,421009-CEDAR-0302,421 - NorthSide,2018-01-01 00:40:00-05:00,Monday,2018-01-01,01,00:40:00,True,0
...,...,...,...,...,...,...,...,...,...
52555,421009-CEDAR-0302,421 - NorthSide,2018-12-31 23:10:00-05:00,Monday,2018-12-31,12,23:10:00,False,0
52556,421009-CEDAR-0302,421 - NorthSide,2018-12-31 23:20:00-05:00,Monday,2018-12-31,12,23:20:00,False,0
52557,421009-CEDAR-0302,421 - NorthSide,2018-12-31 23:30:00-05:00,Monday,2018-12-31,12,23:30:00,False,0
52558,421009-CEDAR-0302,421 - NorthSide,2018-12-31 23:40:00-05:00,Monday,2018-12-31,12,23:40:00,False,0


### Preparing to go through all transactions

In [14]:
bucket_delta = pd.Timedelta(value=10, unit="min")
transactions_2018 = pd.read_csv("../../unholy data dump/2018--transactions.csv")

#extract what we want from the columns and set them as datetime objects
transactions_2018 = transactions_2018[['meter_id', 'zone', 'payment_start_utc', 'payment_end_utc', 'amount']]
transactions_2018['payment_start_utc'] = pd.to_datetime(transactions_2018['payment_start_utc']).dt.tz_localize('UTC')
transactions_2018['payment_end_utc'] = pd.to_datetime(transactions_2018['payment_end_utc']).dt.tz_localize('UTC')

#convert to eastern time
transactions_2018['payment_start_east'] = transactions_2018['payment_start_utc'].dt.tz_convert('US/Eastern')
transactions_2018['payment_end_east'] = transactions_2018['payment_end_utc'].dt.tz_convert('US/Eastern')

#determine how many buckets this car is here for
transactions_2018['start_bucket'] = transactions_2018['payment_start_east'].dt.floor('10min')
transactions_2018['num_buckets'] = (transactions_2018['payment_end_east'] - transactions_2018['start_bucket'])/bucket_delta
transactions_2018['num_buckets'] = transactions_2018['num_buckets'].apply(np.ceil).astype(int)

transactions_2018

,meter_id,zone,payment_start_utc,payment_end_utc,amount,payment_start_east,payment_end_east,start_bucket,num_buckets
0,PBP421,421 - NorthSide,2018-01-01 05:24:00+00:00,2018-01-01 16:59:00+00:00,4.00,2018-01-01 00:24:00-05:00,2018-01-01 11:59:00-05:00,2018-01-01 00:20:00-05:00,70
1,PBP412,412 - East Liberty,2018-01-01 06:10:00+00:00,2018-01-01 14:59:00+00:00,3.00,2018-01-01 01:10:00-05:00,2018-01-01 09:59:00-05:00,2018-01-01 01:10:00-05:00,53
2,PBP421,421 - NorthSide,2018-01-01 06:28:00+00:00,2018-01-01 16:59:00+00:00,4.00,2018-01-01 01:28:00-05:00,2018-01-01 11:59:00-05:00,2018-01-01 01:20:00-05:00,64
3,PBP403-3,403 - Uptown,2018-01-01 06:18:00+00:00,2018-01-01 14:59:00+00:00,3.00,2018-01-01 01:18:00-05:00,2018-01-01 09:59:00-05:00,2018-01-01 01:10:00-05:00,53
4,PBP335,335 - Friendship Cedarville Lot,2018-01-01 11:54:00+00:00,2018-01-01 21:54:00+00:00,9.00,2018-01-01 06:54:00-05:00,2018-01-01 16:54:00-05:00,2018-01-01 06:50:00-05:00,61
...,...,...,...,...,...,...,...,...,...
7740289,341335-18SDNY0002,341 - 18th & Sidney Lot,2019-01-01 02:48:00+00:00,2019-01-01 03:00:00+00:00,1.00,2018-12-31 21:48:00-05:00,2018-12-31 22:00:00-05:00,2018-12-31 21:40:00-05:00,2
7740290,345341-20SDNY0001,345 - 20th & Sidney Lot,2019-01-01 02:33:00+00:00,2019-01-01 03:00:00+00:00,1.00,2018-12-31 21:33:00-05:00,2018-12-31 22:00:00-05:00,2018-12-31 21:30:00-05:00,3
7740291,341334-18SDNY0001,341 - 18th & Sidney Lot,2019-01-01 02:26:00+00:00,2019-01-01 03:00:00+00:00,1.00,2018-12-31 21:26:00-05:00,2018-12-31 22:00:00-05:00,2018-12-31 21:20:00-05:00,4
7740292,344339-18CARSN0001,344 - 18th & Carson Lot,2019-01-01 02:36:00+00:00,2019-01-01 03:00:00+00:00,1.00,2018-12-31 21:36:00-05:00,2018-12-31 22:00:00-05:00,2018-12-31 21:30:00-05:00,3


In [21]:
transactions_by_meter = transactions_2018.groupby('meter_id')
total_elapsed = 0

meter_count = 1
total_meters = len(meter_dict)
unused_meters = 0
for meter in meter_dict:
    try:
        zone = meter_dict[meter]
        folder_name = '../data/meter_occupancies/' + zone.replace('/', '')
        if not os.path.exists(folder_name):
            os.mkdir(folder_name)
        
        transactions = transactions_by_meter.get_group(meter)
        buckets = meter_buckets[meter]
        print("\n*******************************")
        print("Generating for meter: ", meter)
        print("             .....             ")
        t1 = time.time()

        start_index = 0
        for index, transaction in transactions.iterrows():
            # uncomment this line for 7 million + lines of output, or trust that it works
            #print("Proccesing transaction: ", index, " | STARTS: ", transaction['payment_start_east'], " | BUCKETS: ", transaction['num_buckets'])
            start_index = start_index + int((transaction['start_bucket'] - buckets.iloc[start_index,2])/bucket_delta)
            end_index = start_index + transaction['num_buckets']
            buckets.iloc[start_index:end_index, -1] += 1

        meter_buckets[meter] = buckets

        t2 = time.time()
        elapsed = round((t2-t1)/60,2)
        total_elapsed += elapsed

        file_name = folder_name + '/' + meter.replace('/', '') + '.csv'
        print("Completed:    ", meter_count, " / ", total_meters)
        print("Time elapsed: ", elapsed, " minutes")
        print("Creating CSV: ", file_name)
        print("*******************************")
        meter_buckets[meter].to_csv(file_name, index=False)
        meter_count += 1
        
    except KeyError:
        print("\n*******************************")
        print("No transcations for: ", meter)
        print("*******************************")
        meter_count += 1
        unused_meters +=1

print("Bucket generation completed!")
print("Total time: ", total_elapsed, "minutes")
print("Unused meters: ", unused_meters)


*******************************
Generating for meter:  421009-CEDAR-0302
             .....             
Completed:     1  /  1086
Time elapsed:  0.01  minutes
Creating CSV:  ../data/meter_occupancies/421 - NorthSide/421009-CEDAR-0302.csv
*******************************

*******************************
Generating for meter:  409242-NDTHRG0106
             .....             
Completed:     2  /  1086
Time elapsed:  0.03  minutes
Creating CSV:  ../data/meter_occupancies/409 - Oakland 3/409242-NDTHRG0106.csv
*******************************

*******************************
Generating for meter:  PBP412
             .....             
Completed:     3  /  1086
Time elapsed:  3.6  minutes
Creating CSV:  ../data/meter_occupancies/412 - East Liberty/PBP412.csv
*******************************

*******************************
Generating for meter:  426002-CENTRE1802
             .....             
Completed:     4  /  1086
Time elapsed:  0.02  minutes
Creating CSV:  ../data/meter_occupancies/42


*******************************
Generating for meter:  411483-IVYST-0802
             .....             
Completed:     31  /  1086
Time elapsed:  0.19  minutes
Creating CSV:  ../data/meter_occupancies/411 - Shadyside/411483-IVYST-0802.csv
*******************************

*******************************
Generating for meter:  PBP403-3
             .....             
Completed:     32  /  1086
Time elapsed:  2.18  minutes
Creating CSV:  ../data/meter_occupancies/403 - Uptown/PBP403-3.csv
*******************************

*******************************
Generating for meter:  421050-WOHIO-0705
             .....             
Completed:     33  /  1086
Time elapsed:  0.02  minutes
Creating CSV:  ../data/meter_occupancies/421 - NorthSide/421050-WOHIO-0705.csv
*******************************

*******************************
Generating for meter:  419015-BROOKL0708
             .....             
Completed:     34  /  1086
Time elapsed:  0.07  minutes
Creating CSV:  ../data/meter_occupancies

Completed:     61  /  1086
Time elapsed:  0.5  minutes
Creating CSV:  ../data/meter_occupancies/344 - 18th & Carson Lot/344339-18CARSN0001.csv
*******************************

*******************************
Generating for meter:  411482-BELLFN0802
             .....             
Completed:     62  /  1086
Time elapsed:  0.07  minutes
Creating CSV:  ../data/meter_occupancies/411 - Shadyside/411482-BELLFN0802.csv
*******************************

*******************************
Generating for meter:  406606-LIBRTY4904
             .....             
Completed:     63  /  1086
Time elapsed:  0.04  minutes
Creating CSV:  ../data/meter_occupancies/406 - Bloomfield (On-street)/406606-LIBRTY4904.csv
*******************************

*******************************
Generating for meter:  401526-WOODST0404
             .....             
Completed:     64  /  1086
Time elapsed:  0.09  minutes
Creating CSV:  ../data/meter_occupancies/401 - Downtown 1/401526-WOODST0404.csv
************************

Completed:     91  /  1086
Time elapsed:  0.05  minutes
Creating CSV:  ../data/meter_occupancies/406 - Bloomfield (On-street)/406650-PENNAV5103.csv
*******************************

*******************************
Generating for meter:  413040-MURRAY2203
             .....             
Completed:     92  /  1086
Time elapsed:  0.04  minutes
Creating CSV:  ../data/meter_occupancies/413 - Squirrel Hill/413040-MURRAY2203.csv
*******************************

*******************************
Generating for meter:  402420-SMTHFD0101
             .....             
Completed:     93  /  1086
Time elapsed:  0.08  minutes
Creating CSV:  ../data/meter_occupancies/402 - Downtown 2/402420-SMTHFD0101.csv
*******************************

*******************************
Generating for meter:  402418-MARKET0002
             .....             
Completed:     94  /  1086
Time elapsed:  0.06  minutes
Creating CSV:  ../data/meter_occupancies/402 - Downtown 2/402418-MARKET0002.csv
***************************


*******************************
Generating for meter:  407128-TERACE3401
             .....             
Completed:     122  /  1086
Time elapsed:  0.09  minutes
Creating CSV:  ../data/meter_occupancies/407 - Oakland 1/407128-TERACE3401.csv
*******************************

*******************************
Generating for meter:  PBP357
             .....             
Completed:     123  /  1086
Time elapsed:  0.14  minutes
Creating CSV:  ../data/meter_occupancies/357 - Shiloh Street Lot/PBP357.csv
*******************************

*******************************
Generating for meter:  406634-PENNAV4701
             .....             
Completed:     124  /  1086
Time elapsed:  0.01  minutes
Creating CSV:  ../data/meter_occupancies/406 - Bloomfield (On-street)/406634-PENNAV4701.csv
*******************************

*******************************
Generating for meter:  412598-CENTRE5901
             .....             
Completed:     125  /  1086
Time elapsed:  0.03  minutes
Creating CSV:  .

...

Completed:     962  /  1086
Time elapsed:  0.35  minutes
Creating CSV:  ../data/meter_occupancies/321 - Beacon Bartlett Lot/321535-BEBART0003.csv
*******************************

*******************************
Generating for meter:  406617-FRDSHP4701
             .....             
Completed:     963  /  1086
Time elapsed:  0.04  minutes
Creating CSV:  ../data/meter_occupancies/406 - Bloomfield (On-street)/406617-FRDSHP4701.csv
*******************************

*******************************
Generating for meter:  403328-CRAWFD0102
             .....             
Completed:     964  /  1086
Time elapsed:  0.01  minutes
Creating CSV:  ../data/meter_occupancies/403 - Uptown/403328-CRAWFD0102.csv
*******************************

*******************************
Generating for meter:  405026-BUTLER3702
             .....             
Completed:     965  /  1086
Time elapsed:  0.07  minutes
Creating CSV:  ../data/meter_occupancies/405 - Lawrenceville/405026-BUTLER3702.csv
******************

Completed:     992  /  1086
Time elapsed:  0.13  minutes
Creating CSV:  ../data/meter_occupancies/413 - Squirrel Hill/413029-MURRAY1905.csv
*******************************

*******************************
Generating for meter:  413052-PHILPS5802
             .....             
Completed:     993  /  1086
Time elapsed:  0.03  minutes
Creating CSV:  ../data/meter_occupancies/413 - Squirrel Hill/413052-PHILPS5802.csv
*******************************

*******************************
Generating for meter:  422005-ISABEL0001
             .....             
Completed:     994  /  1086
Time elapsed:  0.07  minutes
Creating CSV:  ../data/meter_occupancies/422 - Northshore/422005-ISABEL0001.csv
*******************************

*******************************
Generating for meter:  406628-PENNAV4403
             .....             
Completed:     995  /  1086
Time elapsed:  0.04  minutes
Creating CSV:  ../data/meter_occupancies/406 - Bloomfield (On-street)/406628-PENNAV4403.csv
********************

Completed:     1022  /  1086
Time elapsed:  0.12  minutes
Creating CSV:  ../data/meter_occupancies/408 - Oakland 2/408192-5THAVE4101.csv
*******************************

*******************************
Generating for meter:  421026-ALLGHY0002
             .....             
Completed:     1023  /  1086
Time elapsed:  0.03  minutes
Creating CSV:  ../data/meter_occupancies/421 - NorthSide/421026-ALLGHY0002.csv
*******************************

*******************************
Generating for meter:  405031-BUTLER3901
             .....             
Completed:     1024  /  1086
Time elapsed:  0.01  minutes
Creating CSV:  ../data/meter_occupancies/405 - Lawrenceville/405031-BUTLER3901.csv
*******************************

*******************************
Generating for meter:  413009-FORBES5801
             .....             
Completed:     1025  /  1086
Time elapsed:  0.14  minutes
Creating CSV:  ../data/meter_occupancies/413 - Squirrel Hill/413009-FORBES5801.csv
******************************

Completed:     1052  /  1086
Time elapsed:  0.04  minutes
Creating CSV:  ../data/meter_occupancies/410 - Oakland 4/410177-SCHEDR4705.csv
*******************************

*******************************
Generating for meter:  416002-BVILLE2502
             .....             
Completed:     1053  /  1086
Time elapsed:  0.02  minutes
Creating CSV:  ../data/meter_occupancies/416 - Carrick/416002-BVILLE2502.csv
*******************************

*******************************
Generating for meter:  413038-MURRAY2202
             .....             
Completed:     1054  /  1086
Time elapsed:  0.08  minutes
Creating CSV:  ../data/meter_occupancies/413 - Squirrel Hill/413038-MURRAY2202.csv
*******************************

*******************************
Generating for meter:  421097-EOHIO-0302
             .....             
Completed:     1055  /  1086
Time elapsed:  0.02  minutes
Creating CSV:  ../data/meter_occupancies/421 - NorthSide/421097-EOHIO-0302.csv
*******************************

***

Completed:     1082  /  1086
Time elapsed:  0.23  minutes
Creating CSV:  ../data/meter_occupancies/419 - Brookline/419003-BROOKL0604.csv
*******************************

*******************************
Generating for meter:  413033-MURRAY2003
             .....             
Completed:     1083  /  1086
Time elapsed:  0.1  minutes
Creating CSV:  ../data/meter_occupancies/413 - Squirrel Hill/413033-MURRAY2003.csv
*******************************

*******************************
Generating for meter:  407116-MEYRAN0102
             .....             
Completed:     1084  /  1086
Time elapsed:  0.08  minutes
Creating CSV:  ../data/meter_occupancies/407 - Oakland 1/407116-MEYRAN0102.csv
*******************************

*******************************
Generating for meter:  371566-EOHIOLOT2
             .....             
Completed:     1085  /  1086
Time elapsed:  0.14  minutes
Creating CSV:  ../data/meter_occupancies/371 - East Ohio Street Lot/371566-EOHIOLOT2.csv
**************************

### Now that we've generated the transactions for all meters, we need to begin to address the following:
1. How to disperse mobile transaction meters onto physical meters
2. How to find capacity of a meter

In [5]:
# Load in Zones and Capacities
zone_df = pd.read_csv("../data/zones.csv")
zone_dict = {}

for index, row in zone_df.iterrows():    
    if " - " not in row['zone']:
        continue    
    if row['zone'] in zone_dict:
        zone_dict[row['zone']] = max(zone_dict[row['zone']], row['spaces'])
    else:
        zone_dict[row['zone']] = row['spaces']
        
# Make sure that the file of zonewide occupancies are in the right folder
# You should run all_zones_occupancy first to generate those files if you dont have them.. takes ~3 hours
for zone in zone_dict:
    file_name = '../data/occupancies/' + zone.replace('/','') + '/' + zone.replace('/','') + '.csv'
    if os.path.exists(file_name):
        new_file_name = '../data/zone_occupancies/' + zone.replace('/','') + '.csv'
        shutil.move(file_name, new_file_name)

In [6]:

#This chunk was for me to remove all the index columns because it just gets annoying when writing/reading/writing again
#moral of the story: Always add index=False as a paramter in df.to_csv().

# folder_name = '../data/meter_occupancies/'
# for root, dirs, _ in os.walk(folder_name):
#     for dir in dirs:
#         sub_folder = folder_name + dir
#         for _,_, files in os.walk(sub_folder):
#             for file in files:
#                 file_name = sub_folder + '/' + file
#                 print(file_name)
#                 df = pd.read_csv(file_name)
#                 df = df[['meter','zone','start','day_of_week','date','month', 'time', 'is_holiday','used_spaces']]
#                 df.to_csv(file_name, index=False)


In [60]:
#Lets consider Oakland 1 as a toy example - you can easily modify this for a diff zone and the  rest of the notebook will work
OAKLAND_1 = '407 - Oakland 1'
folder_name = '../data/meter_occupancies/' + OAKLAND_1

mobile_df = None
meter_dfs = {}
for root, dirs, files in os.walk(folder_name):
    for file in files:
        file_name = folder_name + '/' + file
        if file[0] == 'P':
            mobile_df = pd.read_csv(file_name)
            mobile_df.rename(columns={'used_spaces':'mobile_spaces_used'}, inplace=True)
        elif file[0] == 'a':
            #This is the output file of all this business so we don't want to include our output file as a 'meter'
            continue
        else:
            meter = file.split('.')[0]
            meter_dfs[meter] = pd.read_csv(file_name)

mobile_df

,meter,zone,start,day_of_week,date,month,time,is_holiday,mobile_spaces_used
0,PBP410-1,410 - Oakland 4,2018-01-01 00:00:00-05:00,Monday,2018-01-01,1,00:00:00,True,0
1,PBP410-1,410 - Oakland 4,2018-01-01 00:10:00-05:00,Monday,2018-01-01,1,00:10:00,True,0
2,PBP410-1,410 - Oakland 4,2018-01-01 00:20:00-05:00,Monday,2018-01-01,1,00:20:00,True,0
3,PBP410-1,410 - Oakland 4,2018-01-01 00:30:00-05:00,Monday,2018-01-01,1,00:30:00,True,0
4,PBP410-1,410 - Oakland 4,2018-01-01 00:40:00-05:00,Monday,2018-01-01,1,00:40:00,True,0
...,...,...,...,...,...,...,...,...,...
52555,PBP410-1,410 - Oakland 4,2018-12-31 23:10:00-05:00,Monday,2018-12-31,12,23:10:00,False,0
52556,PBP410-1,410 - Oakland 4,2018-12-31 23:20:00-05:00,Monday,2018-12-31,12,23:20:00,False,0
52557,PBP410-1,410 - Oakland 4,2018-12-31 23:30:00-05:00,Monday,2018-12-31,12,23:30:00,False,0
52558,PBP410-1,410 - Oakland 4,2018-12-31 23:40:00-05:00,Monday,2018-12-31,12,23:40:00,False,0


In [61]:
zone_by_meter_df = mobile_df
zone_by_meter_df['metered_spaces_used'] = 0
for meter in meter_dfs:
    df = pd.DataFrame(meter_dfs[meter][['start','used_spaces']])
    df.rename(columns={'used_spaces':meter},inplace=True)
    zone_by_meter_df['metered_spaces_used'] += df[meter]
    zone_by_meter_df = zone_by_meter_df.merge(df, on='start')

zone_by_meter_df

,meter,zone,start,day_of_week,date,month,time,is_holiday,mobile_spaces_used,metered_spaces_used,...,410150-TECHST5004,410142-FREWST4905,410159-SCHEDR3902,410149-TECHST5003,410151-MARMOR5101,410146-FREWST4807,410180-SCHEDR4708,410158-SCHEDR3901,410147-TECHST5001,410148-TECHST5002
0,PBP410-1,410 - Oakland 4,2018-01-01 00:00:00-05:00,Monday,2018-01-01,1,00:00:00,True,0,0,...,0,0,0,0,0,0,0,0,0,0
1,PBP410-1,410 - Oakland 4,2018-01-01 00:10:00-05:00,Monday,2018-01-01,1,00:10:00,True,0,0,...,0,0,0,0,0,0,0,0,0,0
2,PBP410-1,410 - Oakland 4,2018-01-01 00:20:00-05:00,Monday,2018-01-01,1,00:20:00,True,0,0,...,0,0,0,0,0,0,0,0,0,0
3,PBP410-1,410 - Oakland 4,2018-01-01 00:30:00-05:00,Monday,2018-01-01,1,00:30:00,True,0,0,...,0,0,0,0,0,0,0,0,0,0
4,PBP410-1,410 - Oakland 4,2018-01-01 00:40:00-05:00,Monday,2018-01-01,1,00:40:00,True,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52555,PBP410-1,410 - Oakland 4,2018-12-31 23:10:00-05:00,Monday,2018-12-31,12,23:10:00,False,0,0,...,0,0,0,0,0,0,0,0,0,0
52556,PBP410-1,410 - Oakland 4,2018-12-31 23:20:00-05:00,Monday,2018-12-31,12,23:20:00,False,0,0,...,0,0,0,0,0,0,0,0,0,0
52557,PBP410-1,410 - Oakland 4,2018-12-31 23:30:00-05:00,Monday,2018-12-31,12,23:30:00,False,0,0,...,0,0,0,0,0,0,0,0,0,0
52558,PBP410-1,410 - Oakland 4,2018-12-31 23:40:00-05:00,Monday,2018-12-31,12,23:40:00,False,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
for meter in meter_dfs:
    ratio = zone_by_meter_df[meter]/zone_by_meter_df['metered_spaces_used']
    mobile_compensation = ratio * zone_by_meter_df['mobile_spaces_used']
    zone_by_meter_df[meter] = zone_by_meter_df[meter] + mobile_compensation
    zone_by_meter_df[meter] = zone_by_meter_df[meter].fillna(0)

In [63]:
zone_by_meter_df.iloc[198]

meter                                   PBP410-1
zone                             410 - Oakland 4
start                  2018-01-02 09:00:00-05:00
day_of_week                              Tuesday
date                                  2018-01-02
month                                          1
time                                    09:00:00
is_holiday                                 False
mobile_spaces_used                             1
metered_spaces_used                           50
410184-SCHEXT0304                           1.02
410179-SCHEDR4707                              0
410182-SCHEXT0301                              0
410176-SCHEDR4704                              0
410169-SCHEDR4405                              0
410167-SCHEDR4403                              0
410157-MARMOR5108                              0
410177-SCHEDR4705                              0
410185-SCHEDR0301                              0
410183-SCHEXT0302                              0
410172-SCHEDR4602   

In [64]:
for meter in meter_dfs:
    df = meter_dfs[meter]
    df['used_spaces'] = zone_by_meter_df[meter].round(0)

In [65]:
folder_name = '../data/meter_occupancies_with_mobile/' + OAKLAND_1

if not os.path.exists(folder_name):
    os.mkdir(folder_name)

for meter in meter_dfs:
    file_name = folder_name + '/' + meter + '.csv'
    meter_dfs[meter].to_csv(file_name, index=False)

In [51]:
# file_name = folder_name + '/' + 'all_meters.csv'
# zone_by_meter_df.to_csv(file_name, index=False)